# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

In [7]:
import inauguralproject as ip

In [6]:
ip.square(2)

4

# Setting the basics

In [7]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject
# Parameters
rho = 2
v = 0.001
epsilon = 1
omega = 0.5
alpha = 0.5
sigma = 1 
wages_m = 1
wages_f = 1
labour_m = 30
labour_f = 20
hours_m = 12
hours_f = 10

consumption = wages_m*labour_m + wages_f*labour_f
print(consumption)

t_m=labour_m+hours_m
t_f=labour_f+hours_f


def h(hour_m, hour_f, alpha, sigma):
    h[sigma = 0] = np.fmin(hours_m,hours_f)
    h[sigma = 1] = hours_m**(1-alpha)*hours_f**(alpha)
    h[simga != 0,1] =(1-alpha)*hours_m**((sigma-1)/sigma)+alpha*hours_f**((sigma-1)/sigma)
    
    return h

q = consumption**omega*h**(1-omega)





50


# Question 1

**Explain how you solve the model**

In [2]:
# code for solving the model (remember documentation and comments)

a = np.array([1,2,3])
b = inauguralproject.square(a)
print(b)

[1 4 9]


# Question 2

Explain your code and procedure

In [3]:
# code

Comments and conclusion:


# Question 3

Explain your code and procedure

Comments and conclusion

# Question 4

ZeroDivisionError: 0.0 cannot be raised to a negative power

Comments and conclusion

# Question 5

The optimal values of sigma, beta0, and beta1 are: 0.42, 0.50, 0.00
The minimum objective value is: 0.35
The optimal values of sigma_m, sigma_f, beta0, and beta1 are: 0.42, 0.42, 0.50, 0.00
The minimum objective value is: 0.35


Comments and conclusion
